In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from skimage import io, color, transform
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Path to the folders containing real and fake images
original_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
synthetic_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
def load_and_preprocess_images(folder):
    images = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    count=0
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size

            # Convert the NumPy array to a PyTorch tensor
            img_tensor = torch.tensor(img_resized, dtype=torch.float32).to(device)
            images.append(img_tensor)

            count+=1
            print(count)

    return images

In [14]:
fake_images = load_and_preprocess_images(synthetic_folder_path)

Streaming output truncated to the last 5000 lines.
21318
21319
21320
21321
21322
21323
21324
21325
21326
21327
21328
21329
21330
21331
21332
21333
21334
21335
21336
21337
21338
21339
21340
21341
21342
21343
21344
21345
21346
21347
21348
21349
21350
21351
21352
21353
21354
21355
21356
21357
21358
21359
21360
21361
21362
21363
21364
21365
21366
21367
21368
21369
21370
21371
21372
21373
21374
21375
21376
21377
21378
21379
21380
21381
21382
21383
21384
21385
21386
21387
21388
21389
21390
21391
21392
21393
21394
21395
21396
21397
21398
21399
21400
21401
21402
21403
21404
21405
21406
21407
21408
21409
21410
21411
21412
21413
21414
21415
21416
21417
21418
21419
21420
21421
21422
21423
21424
21425
21426
21427
21428
21429
21430
21431
21432
21433
21434
21435
21436
21437
21438
21439
21440
21441
21442
21443
21444
21445
21446
21447
21448
21449
21450
21451
21452
21453
21454
21455
21456
21457
21458
21459
21460
21461
21462
21463
21464
21465
21466
21467
21468
21469
21470
21471
21472
21473
21474
21475
2

In [15]:
real_images = load_and_preprocess_images(original_folder_path)

Streaming output truncated to the last 5000 lines.
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3456
3457
3458
3459
3460
3461
3462
3463
3464
3465
3466
3467
3468
3469
3470
3471
3472
3473
3474
3475
3476
3477
3478
3479
3480
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3491
3492
3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623

In [ ]:
real_images_tensor = torch.stack(real_images)
fake_images_tensor = torch.stack(fake_images)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels_tensor = torch.zeros(len(real_images), dtype=torch.float32).to(device)
fake_labels_tensor = torch.ones(len(fake_images), dtype=torch.float32).to(device)

In [ ]:
# Combine real and fake data
all_images = torch.cat((real_images_tensor, fake_images_tensor), dim=0)
all_labels = torch.cat((real_labels_tensor, fake_labels_tensor), dim=0)

In [ ]:
all_images_numpy = all_images.cpu().numpy()
all_labels_numpy = all_labels.cpu().numpy()

# Split the dataset into training and testing sets
X_train_numpy, X_test_numpy, y_train_numpy, y_test_numpy = train_test_split(
    all_images_numpy, all_labels_numpy, test_size=0.2, random_state=42
)

In [7]:
import cv2

In [8]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []

    for image in images:
        # Perform SIFT feature extraction on CPU
        keypoints, descriptors = sift.detectAndCompute(image, None)

        # Move keypoints and descriptors to GPU (if available)
        keypoints = torch.tensor([kp.pt for kp in keypoints]).to(device)
        descriptors = torch.tensor(descriptors).to(device)

        keypoints_list.append(keypoints)
        descriptors_list.append(descriptors)

    return keypoints_list, descriptors_list

In [ ]:
train_keypoints_list, train_descriptors_list = extract_sift_features(X_train_numpy)
test_keypoints_list, test_descriptors_list = extract_sift_features(X_test_numpy)

In [9]:
def keypoints_to_vectors(keypoints_list, descriptors_list):
    sift_vector_length = 128  # SIFT descriptor length

    features_vectors = []

    for descriptors in descriptors_list:
        if descriptors is None:
            # If no keypoints are detected, add zeros as a placeholder
            features_vectors.append(np.zeros(sift_vector_length))
        else:
            # Randomly choose one descriptor from multiple descriptors
            random_idx = np.random.randint(0, descriptors.shape[0])
            features_vectors.append(descriptors[random_idx])

    # Convert the list of vectors to a PyTorch tensor
    features_vectors = torch.tensor(features_vectors, dtype=torch.float32)

    # Move the tensor to the GPU (if available)
    features_vectors = features_vectors.to(device)

    return features_vectors

In [ ]:
train_fv = keypoints_to_vectors(train_keypoints_list, train_descriptors_list)
test_fv = keypoints_to_vectors(test_keypoints_list, test_descriptors_list)

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
X_train = torch.tensor(train_fv, dtype=torch.float32).to(device)
X_test = torch.tensor(test_fv, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

In [11]:
class CustomModel(nn.Module):
    def __init__(self, input_size):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5)
        )
        self.fc5 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )
        self.fc6 = nn.Sequential(
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        return x

In [ ]:
X_train.shape

NameError: ignored

In [ ]:
# Define the input feature dimension
feature_dim = X_train.shape[1]

# Create an instance of the CustomModel
model = CustomModel(feature_dim)

model.to(device)

NameError: ignored

In [13]:
%pip install torchviz
import torchviz

# Create a sample input tensor with the correct shape
input_tensor = torch.randn(1, 1764)

# Pass the input tensor to the model to generate the architecture
dot = torchviz.make_dot(CustomModel(input_tensor))

# Render and save the model architecture
dot.render("model_architecture.png", format="png")

TypeError: ignored

In [ ]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

In [ ]:
# Define the number of training epochs
num_epochs = 10
batch_size=32

# Convert X_train_features and y_train to PyTorch tensors and create a DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for inputs, labels in train_loader:
        # Move inputs and labels to the GPU if available
        inputs, labels = inputs.to(device), labels.to(device)
        labels=labels.unsqueeze(1)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate and print the average loss for the epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}")

In [ ]:
# Set the model to evaluation mode
model.eval()

# Calculate predictions
with torch.no_grad():
    predictions = model(X_test)

# Calculate the test loss
y_test=y_test.unsqueeze(1)
test_loss = criterion(predictions, y_test)

# Calculate the test accuracy
predictions_binary = (predictions >= 0.5).float()  # Convert probabilities to binary predictions
correct_predictions = (predictions_binary == y_test).sum().item()
total_samples = len(y_test)
test_accuracy = correct_predictions / total_samples

print("Test loss:", test_loss.item())
print("Test accuracy:", test_accuracy)